In [1]:
from pprint import pprint

import xgboost as xgb
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

from mosaicml import *
from mosaicml.constants import MLModelFlavours

/opt/conda/lib/python3.7/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [2]:
diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target
# X, y = load_diabetes(return_X_y=True, as_frame=True)

X_train, X_test, y_train, y_test = train_test_split(X, y)


regressor = xgb.XGBRegressor(n_estimators=10, reg_lambda=1, gamma=0, max_depth=6)

# regressor.fit(X_train, y_train, eval_set=[(X_test, y_test)])
regressor.fit(X_train, y_train)

# from sklearn.ensemble import RandomForestRegressor
# regr = RandomForestRegressor(max_depth=6, random_state=0, n_estimators=10)
# regr.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=10, n_jobs=16,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [3]:
type(X)

numpy.ndarray

In [4]:
type(regressor)

xgboost.sklearn.XGBRegressor

In [5]:
diabetes.feature_names

['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']

In [6]:
def score(model, request):
    payload = request.json["payload"]
    data_list = payload
    data_array = np.asarray(data_list)
    try:
        prediction = model.predict(data_array)
    except:
        prediction = model.predict(data_array.reshape(1, -1))
    return prediction.tolist()

In [7]:
y_pred = regressor.predict(X_test)

In [8]:
# X_train.iloc[0]
# X_test
# X_train[10].tolist()

In [9]:
# to verify if score func works
import requests
req = requests.Request()
req.json = {"payload":X_train[10]}

In [10]:
{"payload":X_train[10]}

{'payload': array([ 0.01628068,  0.05068012,  0.07247433,  0.07695829, -0.00844872,
         0.00557539, -0.00658447, -0.00259226, -0.02364456,  0.06105391])}

In [11]:
print(score(regressor,req))

[133.655517578125]


In [12]:
MLModelFlavours.xgboost

'xgboost'

In [13]:
register_model(regressor,
                score,
                "diabetes_regression_xgboost",
                "diabetes_regression_xgboost",
                MLModelFlavours.xgboost,
                init_script="pip install xgboost==1.5.2",
                input_type="json",
                explain_ai=True,
                x_train=X_train, 
                y_train=y_train, 
                y_true=y_test,
                y_pred=y_pred,
                model_type="regression",
                feature_names=diabetes.feature_names,
                feature_ids=diabetes.feature_names,
                target_names=['var'],
              )

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%
